In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,932 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,347 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,801 kB]
Hit:13 https://ppa.launchpadcontent.net

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_revised = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

home_sales_revised.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_revised.createOrReplaceTempView('home_sales')



In [29]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

spark.sql("""SELECT FORMAT_NUMBER(AVG(price), 2), YEAR(date)
          FROM home_sales WHERE bedrooms == 4
          GROUP BY YEAR(date)
          ORDER BY YEAR(date) DESC""").show()

+----------------------------+----------+
|format_number(avg(price), 2)|year(date)|
+----------------------------+----------+
|                  296,363.88|      2022|
|                  301,819.44|      2021|
|                  298,353.78|      2020|
|                  300,263.70|      2019|
+----------------------------+----------+



In [31]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT FORMAT_NUMBER(AVG(price), 2), YEAR(date)
          FROM home_sales WHERE bedrooms == 3 AND bathrooms == 3
          GROUP BY YEAR(date)
          ORDER BY YEAR(date) DESC""").show()


+----------------------------+----------+
|format_number(avg(price), 2)|year(date)|
+----------------------------+----------+
|                  292,725.69|      2022|
|                  294,211.46|      2021|
|                  294,204.16|      2020|
|                  287,287.82|      2019|
+----------------------------+----------+



In [32]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""SELECT FORMAT_NUMBER(AVG(price), 2), YEAR(date)
          FROM home_sales WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2
          AND sqft_living >= 2000
          GROUP BY YEAR(date)
          ORDER BY YEAR(date) DESC""").show()


+----------------------------+----------+
|format_number(avg(price), 2)|year(date)|
+----------------------------+----------+
|                  290,242.99|      2022|
|                  296,330.96|      2021|
|                  292,289.09|      2020|
|                  289,859.14|      2019|
+----------------------------+----------+



In [34]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
    SELECT FORMAT_NUMBER(AVG(price), 2), view
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------------------------+----+
|format_number(avg(price), 2)|view|
+----------------------------+----+
|                1,026,669.50| 100|
|                1,061,201.42|  99|
|                1,053,739.33|  98|
|                1,129,040.15|  97|
|                1,017,815.92|  96|
|                1,054,325.60|  95|
|                1,033,536.20|  94|
|                1,026,006.06|  93|
|                  970,402.55|  92|
|                1,137,372.73|  91|
|                1,062,654.16|  90|
|                1,107,839.15|  89|
|                1,031,719.35|  88|
|                1,072,285.20|  87|
|                1,070,444.25|  86|
|                1,056,336.74|  85|
|                1,117,233.13|  84|
|                1,033,965.93|  83|
|                1,063,498.00|  82|
|                1,053,472.79|  81|
+----------------------------+----+
only showing top 20 rows

--- 1.4878764152526855 seconds ---


In [35]:
# 7. Cache the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [36]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [37]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
    SELECT FORMAT_NUMBER(AVG(price), 2), view
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------------------------+----+
|format_number(avg(price), 2)|view|
+----------------------------+----+
|                1,026,669.50| 100|
|                1,061,201.42|  99|
|                1,053,739.33|  98|
|                1,129,040.15|  97|
|                1,017,815.92|  96|
|                1,054,325.60|  95|
|                1,033,536.20|  94|
|                1,026,006.06|  93|
|                  970,402.55|  92|
|                1,137,372.73|  91|
|                1,062,654.16|  90|
|                1,107,839.15|  89|
|                1,031,719.35|  88|
|                1,072,285.20|  87|
|                1,070,444.25|  86|
|                1,056,336.74|  85|
|                1,117,233.13|  84|
|                1,033,965.93|  83|
|                1,063,498.00|  82|
|                1,053,472.79|  81|
+----------------------------+----+
only showing top 20 rows

--- 0.6602416038513184 seconds ---


The runtime of the cached table was around 0.66 seconds.  This is less than half the runtime of the uncached table that clocked at around 1.49 seconds.

In [38]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_revised.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [40]:
# 11. Read the parquet formatted data.
parquet_home_sales = spark.read.parquet('home_sales_partitioned')

In [41]:
# 12. Create a temporary table for the parquet data.
parquet_home_sales.createOrReplaceTempView('temp_parquet_home_sales')

In [43]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
    SELECT FORMAT_NUMBER(AVG(price), 2), view
    FROM temp_parquet_home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------------------------+----+
|format_number(avg(price), 2)|view|
+----------------------------+----+
|                1,026,669.50| 100|
|                1,061,201.42|  99|
|                1,053,739.33|  98|
|                1,129,040.15|  97|
|                1,017,815.92|  96|
|                1,054,325.60|  95|
|                1,033,536.20|  94|
|                1,026,006.06|  93|
|                  970,402.55|  92|
|                1,137,372.73|  91|
|                1,062,654.16|  90|
|                1,107,839.15|  89|
|                1,031,719.35|  88|
|                1,072,285.20|  87|
|                1,070,444.25|  86|
|                1,056,336.74|  85|
|                1,117,233.13|  84|
|                1,033,965.93|  83|
|                1,063,498.00|  82|
|                1,053,472.79|  81|
+----------------------------+----+
only showing top 20 rows

--- 0.6899263858795166 seconds ---


After running the previous code twice, the runtime of the parquet was 0.68 seconds.  This was two hundredths of a second longer than the cached table.

In [44]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [45]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False